In [9]:
import os
from typing import List
from datetime import datetime, timezone
import httpx
from pydantic import BaseModel, Field, ConfigDict
from dotenv import load_dotenv
import pandas as pd

# load environment variables
load_dotenv()

# constants
TRANSIT_SECRET = os.getenv("TRANSIT_SECRET", "")
LAT = float(os.getenv("LAT", 0.0))
LON = float(os.getenv("LON", 0.0))
BASE_URL = "https://external.transitapp.com/v3/public/nearby_routes"

In [ ]:
class Route(BaseModel):
    model_config = ConfigDict(extra="ignore")
    id: str = Field(..., alias="global_route_id")
    long_name: str = Field(..., alias="route_long_name")
    short_name: str = Field(..., alias="route_short_name")

class TransitResponse(BaseModel):
    timestamp: str = Field(default_factory=lambda: datetime.now(timezone.utc).isoformat())
    lat: float
    lon: float
    routes: List[Route]

In [11]:
async def fetch_routes(lat: float, lon: float) -> TransitResponse:
    headers = {"apiKey": TRANSIT_SECRET}
    params = {"lat": lat, "lon": lon}
    
    async with httpx.AsyncClient() as client:
        response = await client.get(BASE_URL, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        return TransitResponse(lat=lat, lon=lon, routes=data.get("routes", []))

In [12]:
result = await fetch_routes(LAT, LON)
print(f"Found {len(result.routes)} routes near {LAT}, {LON}")

# display the routes in a dframe
df = pd.DataFrame([r.model_dump() for r in result.routes])
df

Found 5 routes near 51.1, -114.14


,id,long_name,short_name,route_type
0,CAL:1781,N Pointe Term / Foothills Medical Centre,8,3
1,CAL:108838,Nolan Hill,82,3
2,CAL:94353,Dalhousie Station / Lions Park Station,105,3
3,CAL:1790,Dalhousie Station / Chinook Station,9,3
4,CAL:94357,Dalhousie / Montgomery,422,3
